In [1]:
import numpy as np

In [2]:
def make_Mq_from_cAb(c, A, b):
  m, k = A.shape
  m1 = np.hstack((np.zeros((m,m)), -A, b.reshape(m,-1)))
  m2 = np.hstack((A.T, np.zeros((k,k)), -c.reshape(k,-1)))
  m3 = np.append(np.append(-b,c),0)
  M = np.vstack((m1, m2, m3))
  q = np.zeros(m+k+1)
  return M, q

In [3]:
def make_artProb_initialPoint(M,q):
  n, n = M.shape

  x0 = np.ones(n)
  mu0 = np.dot(q,x0)/(n+1)+1
  z0 = mu0/x0
  r = z0 - np.dot(M, x0) - q
  qn1 = (n+1)*mu0

  MM = np.hstack((M, r.reshape((-1,1))))
  MM = np.vstack((MM, np.append(-r,0)))
  qq = np.append(q, qn1)
  xx0 = np.append(x0, 1)
  zz0 = np.append(z0, mu0)
  return MM, qq, xx0, zz0

In [4]:
MEPS = 1.0e-10

def PrimalDualPathFollowing(c, A, b):
  (M0, q0) = make_Mq_from_cAb(c, A, b)
  (M, q, x, z) = make_artProb_initialPoint(M0,q0)
  m, k = A.shape
  n, n = M.shape

  count = 0
  mu = np.dot(z,x)/n
  print('初期目的関数:',mu)
  while mu > MEPS:
    count += 1
    print(count, '回目: ', end=' ')

    delta = 0
    dx = np.dot(np.linalg.inv(M+np.diag(z/x)), delta*mu*(1/x)-z)
    dz = delta*mu*(1/x)-z-np.dot(np.diag(1/x), z*dx)
    th = binarysearch_theta(x, z, dx, dz, 0.5, 0.001)
    print('theta =', th, end=', ')
    x = x + th*dx
    z = z + th*dz
    mu = np.dot(z,x)/n

    delta = 1
    dx = np.dot(np.linalg.inv(M+np.diag(z/x)), delta*mu*(1/x)-z)
    dz = delta*mu*(1/x)-z-np.dot(np.diag(1/x), z*dx)
    x = x + dx
    z = z + dz
    mu = np.dot(z,x)/n
    print('目的関数値:', mu)

  if x[n-2] > MEPS:
    print('Optimal solution:', x[m:m+k]/x[n-2], ' has benn found.')
    print('Optimal value = ', np.dot(c, x[m:m+k]/x[n-2]))
    print('Optimal solution(dual) ', x[:m]/x[n-2], ' has been found.')
    print('Optimal value = ', np.dot(b,x[:m]/x[n-2]))

In [5]:
import numpy as np

In [6]:
def binarysearch_theta(x,z,dx,dz,beta=0.5,precision=0.001):
  n = np.alen(x)

  th_low = 0.0
  th_high = 1.0
  if np.alen(-x[dx<0]/dx[dx<0]) > 0:
    th_high = min(th_high, np.min(-x[dx<0]/dx[dx<0]))
  if np.alen(-z[dz<0]/dz[dz<0]) > 0:
    th_high = min(th_high, np.min(-z[dz<0]/dz[dz<0]))

  x_low = x + th_low*dx
  z_low = z + th_low*dz
  x_high = x + th_high*dx
  z_high = z + th_high*dz
  mu_high = np.dot(x_high, z_high)/n
  if (beta*mu_high >= np.linalg.norm(x_high*z_high - mu_high*np.ones(n))):
    return th_high
  while th_high - th_low > precision:
    th_mid = (th_high + th_low)/2
    x_mid = x + th_mid*dx
    z_mid = z + th_mid*dz
    mu_mid = np.dot(x_mid, z_mid)/n
    if (beta*mu_mid >= np.linalg.norm(x_mid*z_mid - mu_mid*np.ones(n))):
      th_low = th_mid
    else:
      th_high = th_mid

  return th_low

In [7]:
c = np.array([150,200,300])
A = np.array([[3,1,2],[1,3,0],[0,2,4]])
b = np.array([60, 36, 48])

PrimalDualPathFollowing(c, A, b)

初期目的関数: 1.0
1 回目:  theta = 0.5660404758396536, 目的関数値: 0.4339595241603462
2 回目:  theta = 0.5559306039282488, 目的関数値: 0.19270814381346918
3 回目:  theta = 0.6220741646734078, 目的関数値: 0.07282938622494245
4 回目:  theta = 0.692857290584593, 目的関数値: 0.022369015010189928
5 回目:  theta = 0.7495245360789161, 目的関数値: 0.005602889412135011
6 回目:  theta = 0.8709944026483865, 目的関数値: 0.0007228040955075072
7 回目:  theta = 0.9758700546520129, 目的関数値: 1.7441223321897397e-05
8 回目:  theta = 0.998957175794987, 目的関数値: 1.8188129845112275e-08
9 回目:  theta = 0.9990233742220431, 目的関数値: 1.7762996459563537e-11
Optimal solution: [12.  8.  8.]  has benn found.
Optimal value =  5799.999998151829
Optimal solution(dual)  [44.44444443 16.66666666 52.77777776]  has been found.
Optimal value =  5799.999997961326
